# 7. GUI creation

pyqt and pyqtgraph & pymmcore-widgets

In [1]:
import os
import sys
import time
from useq import MDAEvent, MDASequence, Position
from pymmcore_plus.mda import mda_listeners_connected
from pymmcore_plus.mda.handlers import ImageSequenceWriter
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
import numpy as np
from scipy.optimize import curve_fit
import glob
import pandas as pd
import m2stitch
from os import path

In [2]:
# set micro manager directory
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'

# load micro manager system configuration
import os
from pymmcore_plus import CMMCorePlus, Device, DeviceAdapter, ConfigGroup
core = CMMCorePlus()
#core_plus.setDeviceAdapterSearchPaths([mm_dir])
core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))

## pyqt and pyqtgraph <a id='pyqt_and_pyqtgraph'></a>

In [3]:
from PyQt5.QtCore import QTimer
from PyQt5.QtWidgets import QApplication, QLabel, QPushButton, QWidget, QGridLayout
from PyQt5.QtGui import QFont
import pyqtgraph as pg

In [3]:
### simple plot

# start application
app = pg.mkQApp()

# create window
plot_widget = pg.PlotWidget()
x = np.linspace(0, 20, 1000)
y = np.sin(x)
plot_widget.plot(x, y)
plot_widget.show()

# execute application
app.exec()


0

In [ ]:
### moving sine curve ('live data acquisition')

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.t = np.linspace(0, 2*np.pi, 1000)
        self.num = 0.01
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph, 0, 0, 1, 1)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.setGeometry(0, 0, 500, 200)
        self.show()

    def update(self):
        self.graph.plotItem.clear()
        data = np.sin(self.t + self.num)
        self.graph.plotItem.plot(self.t, data)
        self.num = self.num + 0.5

app = QApplication(['auto update'])

win = window()

app.exec_()

In [4]:
### display live microscopy images

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.show()
    
    def update(self):
        core.setConfig('LED_light', 'on')
        self.graph.plotItem.clear()
        core.snapImage()
        im = core.getImage()
        imv = pg.ImageView()
        imv.setImage(im)
        imv.show()
        #imv.close()
        core.setConfig('LED_light', 'off')

app = QApplication(['auto update'])

win = window()

app.exec_()

0

In [1]:
# PyQt5 tutorial
# 

import PyQt5.QtWidgets as qtw
import PyQt5.QtGui as qtg

class MainWindow(qtw.QWidget):
    def __init__(self):
        super().__init__()
        
        # title
        self.setWindowTitle('Microscope Automation')
        
        # layout
        self.setLayout(qtw.QHBoxLayout())
        
        # label
        label = qtw.QLabel('What`s your name?')
        # change font size of label
        label.setFont(qtg.QFont('SansSerif', 30))
        self.layout().addWidget(label)
        
        # create entry box
        entry = qtw.QLineEdit()
        entry.setObjectName('name_field')
        entry.setText('')
        self.layout().addWidget(entry)
        # create a button
        button = qtw.QPushButton('Press me!')
        
        
        self.show()

app = qtw.QApplication([])
mw = MainWindow()

app.exec_()

0

### pymmcore-widgets

In [3]:
# some widgets

from qtpy.QtWidgets import QApplication
from pymmcore_widgets import PropertyBrowser, LiveButton, ExposureWidget, ImagePreview
app = QApplication([])

# create a PropertyBrowser widget. By default, this widget will use the active
# Micro-Manager core instance.

pb_widget = PropertyBrowser()
pb_widget.show()

live_btn = LiveButton()
live_btn.show()

exp_wdg = ExposureWidget()
exp_wdg.show()

img_wdg = ImagePreview()
img_wdg.show()

app.exec_()

0

In [1]:
# preview widget

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus

from qtpy.QtWidgets import QApplication, QGroupBox, QHBoxLayout, QVBoxLayout, QWidget

from pymmcore_widgets import (
    ChannelWidget,
    ExposureWidget,
    ImagePreview,
    LiveButton,
    SnapButton,
    ChannelGroupWidget,
)

class ImageFrame(QWidget):
    """An example widget with a snap/live button and an image preview."""

    def __init__(self):
        super().__init__()

        self.preview = ImagePreview()
        self.snap_button = SnapButton()
        self.live_button = LiveButton()
        self.exposure = ExposureWidget()
        self.channel = ChannelWidget()
        self.channel_group = ChannelGroupWidget()

        self.setLayout(QVBoxLayout())

        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(self.snap_button)
        buttons.layout().addWidget(self.live_button)

        ch_exp = QWidget()
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)
        ch_exp.setLayout(layout)

        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        ch.layout().setContentsMargins(0, 0, 0, 0)
        ch.layout().addWidget(self.channel)
        layout.addWidget(ch)

        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        ch_gr.layout().setContentsMargins(0, 0, 0, 0)
        ch_gr.layout().addWidget(self.channel_group)
        layout.addWidget(ch_gr)

        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        exp.layout().setContentsMargins(0, 0, 0, 0)
        exp.layout().addWidget(self.exposure)
        layout.addWidget(exp)

        self.layout().addWidget(self.preview)
        self.layout().addWidget(ch_exp)
        self.layout().addWidget(buttons)


if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    #core.loadSystemConfiguration()
    app = QApplication([])
    frame = ImageFrame()
    frame.show()
    #core.snap()
    app.exec_()

C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in sca

![Preview Window](preview_window.png "Title")

In [2]:
core.setConfig('LED_light', 'off')

In [1]:
# multiple widgets in main window

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus
import PyQt5.QtWidgets as qtw
from qtpy.QtWidgets import QStackedWidget, QMainWindow, QAction, QApplication, QGroupBox, QWidget
import pymmcore_widgets as pycw

class MainWindow(qtw.QWidget):
    def __init__(self):
        super().__init__()
        
        self.setLayout(qtw.QHBoxLayout())
        
        snap_button = pycw.SnapButton()
        live_button = pycw.LiveButton()
        image_preview = pycw.ImagePreview()
        channel_group = pycw.ChannelGroupWidget()
        channel = pycw.ChannelWidget()
        
        buttons = QGroupBox()
        buttons.setLayout(qtw.QHBoxLayout())
        buttons.layout().addWidget(image_preview)
        buttons.layout().addWidget(snap_button)
        buttons.layout().addWidget(live_button)
        buttons.layout().addWidget(channel_group)
        buttons.layout().addWidget(channel)
        
        stage_widget = pycw.StageWidget('XYStage')
        
        stage = QGroupBox()
        stage.setLayout(qtw.QHBoxLayout())
        stage.layout().addWidget(stage_widget)
        
        self.layout().addWidget(buttons)
        self.layout().addWidget(stage)
        
        self.show()

if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    app = qtw.QApplication([])
    mw = MainWindow()
    app.exec_()

C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in sca

![Preview Window + stage](preview_window_stage.png)

In [2]:
core.setConfig('LED_light', 'off')

In [1]:
import PyQt5.QtWidgets as qtw
from qtpy.QtWidgets import QApplication, QGroupBox, QHBoxLayout, QVBoxLayout, QWidget
import pymmcore_widgets as pycw
import PyQt5.QtGui as qtg
import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus

class MainWindow(qtw.QWidget):
    def __init__(self):
        super().__init__()
        
        # image preview
        preview_window = QWidget()
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)
        preview_window.setLayout(layout)
        
        self.preview = pycw.ImagePreview()
        self.snap_button = pycw.SnapButton()
        self.live_button = pycw.LiveButton()
        self.exposure = pycw.ExposureWidget()
        self.channel = pycw.ChannelWidget()
        self.channel_group = pycw.ChannelGroupWidget()

        self.setLayout(QVBoxLayout())

        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(self.snap_button)
        buttons.layout().addWidget(self.live_button)

        ch_exp = QWidget()
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)
        ch_exp.setLayout(layout)

        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        ch.layout().setContentsMargins(0, 0, 0, 0)
        ch.layout().addWidget(self.channel)
        layout.addWidget(ch)

        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        ch_gr.layout().setContentsMargins(0, 0, 0, 0)
        ch_gr.layout().addWidget(self.channel_group)
        layout.addWidget(ch_gr)

        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        exp.layout().setContentsMargins(0, 0, 0, 0)
        exp.layout().addWidget(self.exposure)
        layout.addWidget(exp)

        self.layout().addWidget(self.preview)
        self.layout().addWidget(ch_exp)
        self.layout().addWidget(buttons)
        
        self.preview_window().addWidget(layout)
    
        self.show()

if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    app = qtw.QApplication([])
    mw = MainWindow()

    app.exec_()

TypeError: 'QGroupBox' object is not callable